In [1]:
from io import open
!pip install conllu
from conllu import parse_incr
from gensim.models import Word2Vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
from torch import nn, optim

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import nltk.corpus


In [5]:
def load_data(file_name):
    data_file = open(file_name, "r", encoding="utf-8")
    # get 0th, 2nd and 4th columns of different sentences
    sentences = []
    sentence_tags = []
    for tokenlist in parse_incr(data_file):
        curr_sentence = []
        curr_tags = []
        for token in tokenlist:
            sen=token["form"]
            curr_sentence.append(sen)
            tag=token["upostag"] 
            curr_tags.append(tag)
        sentences.append(curr_sentence)
        sentence_tags.append(curr_tags)
    return sentences, sentence_tags
train_filepath="/content/drive/MyDrive/NLP_ASSG2/en_atis-ud-train.conllu"
test_filepath="/content/drive/MyDrive/NLP_ASSG2/en_atis-ud-test.conllu"
train_sentences, train_tags = load_data(train_filepath)
test_sentences, test_tags = load_data(test_filepath)

In [6]:
import nltk

class DependencyParser:
    def __init__(self, grammar):
        self.grammar = grammar
    
    def parse(self, sentence):
        tokens = nltk.word_tokenize(sentence)
        parser = nltk.parse.ShiftReduceParser(self.grammar)
        tree = None
        
        try:
            for tree in parser.parse(tokens):
                pass
        except ValueError:
            return None
        
        if tree is None:
            return None
        
        dependencies = []
        for node in tree.nodes.values():
            if node["address"] == 0:
                continue
            governor = tokens[node["head"]]
            dependent = tokens[node["address"] - 1]
            dependencies.append((governor, dependent, node["rel"]))
        
        return dependencies


In [12]:
grammar = nltk.DependencyGrammar.fromstring("""
'saw' -> 'I' | 'her' | 'the' | 'with'
'with' -> 'telescope'
'telescope' -> 'my'
""")


In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
parser = DependencyParser(grammar)
sentence = "I saw her with my telescope"
dependencies = parser.parse(sentence)
print(dependencies)


[('saw', 'I', 'nsubj'), ('saw', 'her', 'dobj'), ('saw', 'telescope', 'prep'), ('telescope', 'my', 'poss')]
